<a href="https://colab.research.google.com/github/sriyamistry0-lang/CRISPR_Colours/blob/main/Site_Science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install biopython
import pandas as pd
from Bio.Seq import Seq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 50.3 MB/s eta 0:00:00


In [4]:
# Sample Code for trial
# 1. Paste your target gene sequence here (Example: first 200bp of gyrA)
gene_seq = "ATGAGCGACCTTGCGAGAGAAATTACACCGGTCAACATTGAGGAAGAGCTGAAGAGCTCCTATCTGGATTATGCGATGTCGGTCATTGTTGGCCGTGCGCTGCCAGATGTCCGAGACGGCCTGAAGCCGGTACACCGTCGCGTACTTTACGCCATGAACGTACTTGGTGGCTGGCAGGCATTAGTGGGGTTG"

def design_crispr_attack(sequence):
    results = []
    # Search for PAM (NGG)
    for i in range(len(sequence) - 23):
        sub_seq = sequence[i:i+23]
        pam = sub_seq[20:]

        if pam.endswith("GG"):  # Looking for NGG
            spacer = sub_seq[:20]

            # DESIGNING THE OLIGOS FOR pCas9 (BsaI Overhangs)
            # Top: 5'-GAAA + Spacer -3'
            # Bottom: 5'-AAAC + Reverse Complement of Spacer -3'
            top_oligo = "GAAA" + spacer
            bottom_oligo = "AAAC" + str(Seq(spacer).reverse_complement())

            results.append({
                "Position": i,
                "Spacer (20bp)": spacer,
                "PAM": pam,
                "Top Oligo (Order this)": top_oligo,
                "Bottom Oligo (Order this)": bottom_oligo
            })
    return pd.DataFrame(results)

df = design_crispr_attack(gene_seq)
df.head(10) # Shows the top 10 attack sites

,Position,Spacer (20bp),PAM,Top Oligo (Order this),Bottom Oligo (Order this)
0,8,CCTTGCGAGAGAAATTACAC,CGG,GAAACCTTGCGAGAGAAATTACAC,AAACGTGTAATTTCTCTCGCAAGG
1,20,AATTACACCGGTCAACATTG,AGG,GAAAAATTACACCGGTCAACATTG,AAACCAATGTTGACCGGTGTAATT
2,44,AGAGCTGAAGAGCTCCTATC,TGG,GAAAAGAGCTGAAGAGCTCCTATC,AAACGATAGGAGCTCTTCAGCTCT
3,59,CTATCTGGATTATGCGATGT,CGG,GAAACTATCTGGATTATGCGATGT,AAACACATCGCATAATCCAGATAG
4,69,TATGCGATGTCGGTCATTGT,TGG,GAAATATGCGATGTCGGTCATTGT,AAACACAATGACCGACATCGCATA
5,96,GCGCTGCCAGATGTCCGAGA,CGG,GAAAGCGCTGCCAGATGTCCGAGA,AAACTCTCGGACATCTGGCAGCGC
6,107,TGTCCGAGACGGCCTGAAGC,CGG,GAAATGTCCGAGACGGCCTGAAGC,AAACGCTTCAGGCCGTCTCGGACA
7,144,CTTTACGCCATGAACGTACT,TGG,GAAACTTTACGCCATGAACGTACT,AAACAGTACGTTCATGGCGTAAAG
8,147,TACGCCATGAACGTACTTGG,TGG,GAAATACGCCATGAACGTACTTGG,AAACCCAAGTACGTTCATGGCGTA
9,151,CCATGAACGTACTTGGTGGC,TGG,GAAACCATGAACGTACTTGGTGGC,AAACGCCACCAAGTACGTTCATGG


In [5]:
!pip install biopython
import pandas as pd
from Bio.Seq import Seq

print("--- CRISPR-Cas9 Attack-Bot: Plasmid Design Tool ---")
print("Follow the prompts to design your custom 'Killer' plasmid.\n")

# --- USER INPUT SECTION ---

# 1. Plasmid Name
plasmid_name = input("Enter the name of your backbone plasmid (e.g., pCas9): ") or "pCas9"

# 2. Restriction Enzyme Sites (For Golden Gate / BsaI)
# Standard pCas9/BsaI overhangs are usually GAAA and AAAC
overhang_top = input("Enter the 5' overhang for the TOP oligo (Example: GAAA): ") or "GAAA"
overhang_bottom = input("Enter the 5' overhang for the BOTTOM oligo (Example: AAAC): ") or "AAAC"

# 3. The Target Gene Sequence
print("\n--- TARGET GENE ENTRY ---")
example_gyra = "ATGAGCGACCTTGCGAGAGAAATTACACCGGTCAACATTGAGGAAGAGCTGAAGAGCTCCTATCTGGATTATGCGATGTCGGTCATTGTTGGCCGTGCGCTGCCAGATGTCCGAGACGGCCTGAAGCCGGTACACCGTCGCGTACTTTACGCCATGAACGTACTTGGTGGCTGGCAGGCATTAGTGGGGTTG"
print(f"Example Sequence (gyrA fragment): \n{example_gyra}\n")

target_seq = input("Paste your E. coli gene sequence (Press Enter to use the example above): ")
if not target_seq:
    target_seq = example_gyra

# --- PROCESSING LOGIC ---

def design_attack_oligos(sequence, p_name, oh_top, oh_bottom):
    results = []
    # Clean sequence of any spaces or newlines
    clean_seq = "".join(sequence.split()).upper()

    # We search for the PAM (NGG) on the Sense strand
    # The spacer is the 20bp BEFORE the NGG
    for i in range(len(clean_seq) - 23):
        sub_seq = clean_seq[i:i+23]
        pam = sub_seq[20:]

        if pam.endswith("GG"):  # Logic: Cas9 looks for NGG
            spacer = sub_seq[:20]

            # The DNA we insert must result in a Sense RNA gRNA.
            # To do that, the plasmid transcribes the 'spacer' we provide.
            # We add the user-defined overhangs for the BsaI sticky ends.

            top_oligo = oh_top + spacer
            # Bottom oligo is the reverse complement + the matching overhang
            bottom_oligo = oh_bottom + str(Seq(spacer).reverse_complement())

            results.append({
                "Target Site": f"{p_name}_site_{i}",
                "Position": i,
                "Spacer_20bp": spacer,
                "PAM": pam,
                "TOP_Oligo_5_3": top_oligo,
                "BOTTOM_Oligo_5_3": bottom_oligo
            })

    return pd.DataFrame(results)

# --- EXECUTION & CLEAN RESULTS ---

df = design_attack_oligos(target_seq, plasmid_name, overhang_top, overhang_bottom)

if df.empty:
    print("\n[!] No valid PAM (NGG) sites found in that sequence. Try a longer segment!")
else:
    print(f"\n--- SUCCESS: Found {len(df)} potential attack sites in {plasmid_name} ---")
    # Displaying first 10 results for clarity
    print(df[['Position', 'Spacer_20bp', 'PAM', 'TOP_Oligo_5_3', 'BOTTOM_Oligo_5_3']].head(10))

    # Optional: Save to CSV for easy ordering
    df.to_csv("crispr_attack_oligos.csv", index=False)
    print("\nResults saved to 'crispr_attack_oligos.csv'. You can download this from the folder icon on the left.")

--- CRISPR-Cas9 Attack-Bot: Plasmid Design Tool ---
Follow the prompts to design your custom 'Killer' plasmid.

Enter the name of your backbone plasmid (e.g., pCas9): pCas9
Enter the 5' overhang for the TOP oligo (Example: GAAA): GAAA
Enter the 5' overhang for the BOTTOM oligo (Example: AAAC): AAAC

--- TARGET GENE ENTRY ---
Example Sequence (gyrA fragment): 
ATGAGCGACCTTGCGAGAGAAATTACACCGGTCAACATTGAGGAAGAGCTGAAGAGCTCCTATCTGGATTATGCGATGTCGGTCATTGTTGGCCGTGCGCTGCCAGATGTCCGAGACGGCCTGAAGCCGGTACACCGTCGCGTACTTTACGCCATGAACGTACTTGGTGGCTGGCAGGCATTAGTGGGGTTG

Paste your E. coli gene sequence (Press Enter to use the example above): ATGAGCGACCTTGCGAGAGAAATTACACCGGTCAACATTGAGGAAGAGCTGAAGAGCTCCTATCTGGATTATGCGATGTCGGTCATTGTTGGCCGTGCGCTGCCAGATGTCCGAGACGGCCTGAAGCCGGTACACCGTCGCGTACTTTACGCCATGAACGTACTTGGTGGCTGGCAGGCATTAGTGGGGTTG

--- SUCCESS: Found 14 potential attack sites in pCas9 ---
   Position           Spacer_20bp  PAM             TOP_Oligo_5_3  \
0         8  CCTTGCGAGAGAAATTACAC  CGG  GAAACCTTGCG